## Training and logging our model

In [2]:
!pip install mlflow

     |████████████████████████████████| 14.2 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 370 kB 68.9 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.2 MB/s  eta 0:00:01
     |████████████████████████████████| 146 kB 69.6 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 42.7 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 48.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 61 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 298 kB 68.2 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.2 MB/s  eta 0:00:01
  Created wheel for gunicorn: filename=gunicorn-20.1.0-py3-none-any.whl size=78918 sha256=f5a71c0b2f8a17e47e7503305a7f6d9a2d54d4ae311b4ade8348ece30d6c4384
  Stored in directory: /home/jovyan/.cache/pip/wheels/21/4b/32/9be8daf8a4d73da26e4dba66c47c

In [3]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

iris = load_iris()

X = pd.DataFrame(data = iris["data"], columns= iris["feature_names"])
y = pd.DataFrame(data = iris["target"], columns=["target"])

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
with mlflow.start_run():
    # Specified Parameters 
    c = 0.5

    # Instanciate and fit the model 
    lr = LogisticRegression(C=c)
    lr.fit(X_train.values, y_train.values)

    # Store metrics 
    predicted_qualities = lr.predict(X_test.values)
    accuracy = lr.score(X_test.values, y_test.values)

    # Print results 
    print("LogisticRegression model")
    print("Accuracy: {}".format(accuracy))

    # Log Metric 
    mlflow.log_metric("Accuracy", accuracy)

    # Log Param
    mlflow.log_param("C", c)
    
    # Log model
    mlflow.sklearn.log_model(lr, "model")

LogisticRegression model
Accuracy: 0.9473684210526315


/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## Deploying our model to Sagemaker

In [ ]:
app_name = "iris"

# Region where you want your model to be deployed
region = "eu-west-3"

# You can change the instance type
instance_type = "ml.t2.medium"

# Get it from Mlflow
run_id = ""
model_uri = "runs:/" + run_id + "/model"

# Put your AWS Role ARN
execution_role = ""

# Put the image URI took from AWS ECR
image_ecr_url = ""

In [ ]:
# Create a SageMaker app 
import mlflow.sagemaker as mfs

mfs.deploy(
    app_name=app_name,
    model_uri=model_uri,
    image_url=image_ecr_url,
    region_name=region,
    mode="replace",
    execution_role_arn=execution_role,
    instance_type=instance_type
)

## Test our model endpoint

In [ ]:
# Format input for API 
input_json = X_test.to_json(orient="split")
input_json

In [ ]:
# Use the API endpoint
import json
import boto3

def query_endpoint(app_name, input_json):
    # Instanciate a client in order to access Sagemaker
    client = boto3.session.Session().client("sagemaker-runtime", region)
    # Call our Iris endpoint
    response = client.invoke_endpoint(
        EndpointName=app_name,
        Body=input_json,
        ContentType='application/json; format=pandas-split',
    )
    # We get the answer
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    print("Received response: {}".format(preds))

    return preds

In [ ]:
# Evaluate the input by posting it to the deployed model
prediction1 = query_endpoint(app_name=app_name, input_json=input_json)

In [ ]:
# Delete app 
import mlflow.sagemaker as mfs

# Specify the archive=False option to delete any SageMaker models and configurations
# associated with the specified application
mfs.delete(app_name=app_name, region_name=region, archive=False)